In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


###PCA analysis and visualisation demo.

``Run time``: approximately ? minutes (workstation circa 2010)

``Memory requirement``: ~ ?GB

In [4]:
from tvb.datatypes.cortex import Cortex

from tvb.simulator.lab import *
from tvb.datatypes.time_series import TimeSeriesSurface
from tvb.simulator.plot import timeseries_interactive as timeseries_interactive
from tvb.datatypes.local_connectivity import LocalConnectivity
import tvb.analyzers.pca as pca

In [5]:
# if the demo_data_surface_8.125s_2048HZ.npy file is not present, this function will generate it.
def generate_surface_demo_data():
    """
    Generate 8.125 seconds of 2048 Hz data at the surface level, stochastic integration.

    ``Run time``: approximately 5 hours (workstation circa 2010, MKL.)

    ``Memory requirement``: ~ 7 GB
    ``Storage requirement``: 2.1 GB

    .. moduleauthor:: Stuart A. Knock <stuart.knock@gmail.com>

    """

    ##----------------------------------------------------------------------------##
    ##-                      Perform the simulation                              -##
    ##----------------------------------------------------------------------------##
    
    #TODO: Configure this so it actually generates an interesting timeseries.
    #      Start with a local_coupling that has a broader footprint than the default.
    
    LOG.info("Configuring...")
    #Initialise a Model, Coupling, and Connectivity.
    oscillator = models.Generic2dOscillator()
    
    white_matter = connectivity.Connectivity(load_default=True)
    white_matter.speed = numpy.array([4.0])
    white_matter_coupling = coupling.Linear(a=2 ** -9)
    
    #Initialise an Integrator
    hiss = noise.Additive(nsig=numpy.array([2 ** -16, ]))
    heunint = integrators.HeunStochastic(dt=0.06103515625, noise=hiss)
    #TODO: Make dt as big as possible to shorten runtime; restrictions of integral multiples to get desired monitor period
    
    #Initialise a Monitor with period in physical time
    what_to_watch = monitors.TemporalAverage(period=0.48828125)     # 2048Hz => period=1000.0/2048.0
    
    #Initialise a surface
    local_coupling_strength = numpy.array([0.0115])
    
    grey_matter = LocalConnectivity(load_default=True)
    grey_matter.cutoff = 60.0
    grey_matter.equation.parameters['sigma1'] = 10.0
    grey_matter.equation.parameters['sigma2'] = 20.0
    grey_matter.equation.parameters['amp1'] = 1.0
    grey_matter.equation.parameters['amp2'] = 0.5
    
    default_cortex = Cortex.from_file()
    default_cortex.local_connectivity = grey_matter
    default_cortex.coupling_strength = local_coupling_strength
    
    #Initialise a Simulator -- Model, Connectivity, Integrator, and Monitors.
    sim = simulator.Simulator(model=oscillator, connectivity=white_matter,
                              coupling=white_matter_coupling,
                              integrator=heunint, monitors=what_to_watch,
                              surface=default_cortex)
    sim.configure()
    
    #Clear initial transient
    LOG.info("Initial run to clear transient...")
    for _ in sim(simulation_length=125):
        pass
    LOG.info("Finished initial run to clear transient.")
    
    
    #Perform the simulation
    tavg_data = []
    tavg_time = []
    LOG.info("Starting simulation...")
    for tavg in sim(simulation_length=8125):
        if not tavg is None:
            tavg_time.append(tavg[0][0])
            tavg_data.append(tavg[0][1])
    
    LOG.info("Finished simulation.")
    
    
    ##----------------------------------------------------------------------------##
    ##-                     Save the data to a file                              -##
    ##----------------------------------------------------------------------------##
    
    #Make the list a numpy.array.
    LOG.info("Converting result to array...")
    TAVG = numpy.array(tavg_data)
    
    #Save it
    FILE_NAME = "demo_data_surface_8.125s_2048Hz.npy"
    LOG.info("Saving array to %s..." % FILE_NAME)
    numpy.save(FILE_NAME, TAVG)
    
    LOG.info("Done.")
    
    ###EoF###

In [6]:
#Load the demo surface timeseries dataset 
try:
    data = numpy.load("demo_data_surface_8.125s_2048Hz.npy")
except IOError:
    LOG.error("Can't load demo data. Run demos/generate_surface_demo_data.py")
    generate_surface_demo_data()



In [7]:
period = 0.00048828125  # s

In [8]:
#Initialse a default surface
default_cortex = Cortex.from_file()


In [ ]:
#Put the data into a TimeSeriesSurface datatype
tsr = TimeSeriesSurface(surface=default_cortex,
                        data=data,
                        sample_period=period)
tsr.configure()

In [ ]:
#Create and run the analyser
pca_analyser = pca.PCA(time_series=tsr)
pca_data = pca_analyser.evaluate()

In [ ]:
#Generate derived data, such as, component time series, etc.
pca_data.configure()

In [ ]:
#Put the data into a TimeSeriesSurface datatype
component_tsr = TimeSeriesSurface(surface=default_cortex,
                                  data=pca_data.component_time_series,
                                  sample_period=period)
component_tsr.configure()


In [ ]:
#Prutty pictures...
tsi = timeseries_interactive.TimeSeriesInteractive(time_series=component_tsr,
                                                   first_n=16)
tsi.configure()
tsi.show()